In [85]:
import numpy as np
matrix = np.array([
    [6.20, 1.10, 0.94, 1.21],
    [1.10, 4.10, 1.30, 0.16],
    [0.94, 1.30, 7.40, 1.10],
    [1.21, 0.16, 1.10, 9.10]
])

In [86]:
# matrix = np.array([
#     [2.2, 1, 0.5, 2],
#     [1, 1.3, 2, 1],
#     [0.5, 2, 0.5, 1.6],
#     [2, 1, 1.6, 2]
# ])
# matrix

In [87]:
def latex_matrix(matrix):
    matrix_ = matrix.copy()
    text = "$\n"
    text += r"\left(\begin{matrix}" + "\n"
    for row in matrix_:
        text += " "*4 + " & ".join(map(str, row)) + r" \\" + "\n"
    text += r"\end{matrix}\right)"
    text += "\n$"
    return text
print(latex_matrix(matrix))

$
\left(\begin{matrix}
    6.2 & 1.1 & 0.94 & 1.21 \\
    1.1 & 4.1 & 1.3 & 0.16 \\
    0.94 & 1.3 & 7.4 & 1.1 \\
    1.21 & 0.16 & 1.1 & 9.1 \\
\end{matrix}\right)
$


In [88]:
def Danilevsky(matrix):
    n = len(matrix)
    Ps = np.zeros((n, n, n))
    Ps[-1] = matrix.copy()
    Ms = np.array(list(np.identity(n)for i in range(n-1)))
    Ms_1 = np.array(list(np.identity(n)for i in range(n-1)))
    M = None
    for k in range(n-2, -1, -1):
    # print(Ps[k+1][k+1])
    # print(Ps[k+1][k+1][k])
        Ms[k][k] = -Ps[k+1][k+1] / Ps[k+1][k+1][k]
        Ms[k][k][k] = -Ms[k][k][k] / Ps[k+1][k+1][k]

        Ms_1[k][k] = Ps[k+1][k+1]
        index = k + 1
        print(f"$A_{index} = $", latex_matrix(np.round(Ps[k+1], 4)) + "\n", r"$M^{-1}" + f"_{index} = $", latex_matrix(np.round(Ms_1[k], 4)) + "\n", f"$M_{index} = $", latex_matrix(np.round(Ms[k], 4)) + "\n", sep='\n')

        Ps[k] = Ms_1[k] @ Ps[k+1] @ Ms[k]
        if M is None:
            M = Ms[k]
        else:
            M @= Ms[k]

    print(f"$A_{0} = P = $", latex_matrix(np.round(Ps[0], 4)) + "\n", sep='\n')
    # print(Ms[k])
    # print(Ms_1[k])
    # print(Ps[k])

    return Ps[0][0], M

coefs, M = Danilevsky(matrix)
coefs, M

$A_3 = $
$
\left(\begin{matrix}
    6.2 & 1.1 & 0.94 & 1.21 \\
    1.1 & 4.1 & 1.3 & 0.16 \\
    0.94 & 1.3 & 7.4 & 1.1 \\
    1.21 & 0.16 & 1.1 & 9.1 \\
\end{matrix}\right)
$

$M^{-1}_3 = $
$
\left(\begin{matrix}
    1.0 & 0.0 & 0.0 & 0.0 \\
    0.0 & 1.0 & 0.0 & 0.0 \\
    1.21 & 0.16 & 1.1 & 9.1 \\
    0.0 & 0.0 & 0.0 & 1.0 \\
\end{matrix}\right)
$

$M_3 = $
$
\left(\begin{matrix}
    1.0 & 0.0 & 0.0 & 0.0 \\
    0.0 & 1.0 & 0.0 & 0.0 \\
    -1.1 & -0.1455 & 0.9091 & -8.2727 \\
    0.0 & 0.0 & 0.0 & 1.0 \\
\end{matrix}\right)
$

$A_2 = $
$
\left(\begin{matrix}
    5.166 & 0.9633 & 0.8545 & -6.5664 \\
    -0.33 & 3.9109 & 1.1818 & -10.5945 \\
    -1.7219 & 2.0373 & 17.7231 & -75.7704 \\
    0.0 & -0.0 & 1.0 & 0.0 \\
\end{matrix}\right)
$

$M^{-1}_2 = $
$
\left(\begin{matrix}
    1.0 & 0.0 & 0.0 & 0.0 \\
    -1.7219 & 2.0373 & 17.7231 & -75.7704 \\
    0.0 & 0.0 & 1.0 & 0.0 \\
    0.0 & 0.0 & 0.0 & 1.0 \\
\end{matrix}\right)
$

$M_2 = $
$
\left(\begin{matrix}
    1.0 & 0.0 & 0.0 & 0.0

(array([   26.8       ,  -256.2267    ,  1031.97237   , -1465.66094076]),
 array([[ -0.23610292,   4.91562432, -30.62682133,  52.97308101],
        [ -0.19955528,   4.64555284, -34.5852113 ,  81.96458416],
        [  0.28873943,  -6.08290353,  39.62917056, -78.46523771],
        [  0.        ,   0.        ,   0.        ,   1.        ]]))

In [89]:
print(latex_matrix(np.atleast_2d(np.round(coefs, 4)).T))

$
\left(\begin{matrix}
    26.8 \\
    -256.2267 \\
    1031.9724 \\
    -1465.6609 \\
\end{matrix}\right)
$


In [90]:
P = np.round(np.linalg.inv(M) @ matrix @ M, 6)
print(latex_matrix(P))
P

$
\left(\begin{matrix}
    26.8 & -256.2267 & 1031.97237 & -1465.660941 \\
    1.0 & 0.0 & -0.0 & 0.0 \\
    -0.0 & 1.0 & -0.0 & 0.0 \\
    -0.0 & 0.0 & 1.0 & -0.0 \\
\end{matrix}\right)
$


array([[ 2.68000000e+01, -2.56226700e+02,  1.03197237e+03,
        -1.46566094e+03],
       [ 1.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         0.00000000e+00],
       [-0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
         0.00000000e+00],
       [-0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -0.00000000e+00]])

In [91]:
def F(X, coefs):
    n = len(coefs)
    degrees = np.arange(n-1, -1, -1)
    # print(X, n, coefs, degrees)
    # [-2, 1, 5, -2, 7]
    # f = -2*x**4 + x**3 + 5*x**2 - 2*x + 7
    res = np.zeros(len(X))
    for x_index in range(len(X)):
        for degree_index in range(len(degrees)):
            res[x_index] += coefs[degree_index] * X[x_index]**degrees[degree_index]
    return res

def find_intervals(coefs, interval_of_finding=[-10, 10], dots_count=30):
    dots_count = int(dots_count)
    intervals = []
    n = len(coefs)
    polys = [coefs, np.polyder(coefs)]
    for i in range(1, n-1):
        polys.append(-np.polydiv(polys[i-1], polys[i])[1])

    # print(polys)

    # X = np.array([-2, -1.7, -1.4, -1, 0.8, 1.1, 1.4, 1.7, 2.0, 2.1])
    X = np.linspace(*interval_of_finding, dots_count)
    matrix = np.zeros((len(polys), len(X)))
    for poly_index in range(len(polys)):
        # print(polys[poly_index])
        # matrix[poly_index] = F(X, polys[poly_index])
        matrix[poly_index] = np.polyval(polys[poly_index], X)

    # print(matrix.T)
    # print(np.sum(matrix.T > 0, axis=1))
    # print(np.sign(matrix.T))
    # print(np.diff(np.sign(matrix.T), axis=1) != 0)
    sign_change = np.sum(np.diff(np.sign(matrix.T), axis=1) != 0, axis=1)
    difference = np.diff(sign_change, axis=0) != 0
    for diff_index in range(len(difference)):
        if difference[diff_index]:
            intervals.append([X[diff_index], X[diff_index+1]])
    return intervals
    # print(difference)
coef = np.array([1, *(-coefs)])
# print(coef)
intervals = find_intervals(coef, interval_of_finding=[-100, 100], dots_count=1e3)
intervals
# find_intervals(np.array([-2, 1, 5, -2, 7]), interval_of_finding=[-10, 10], dots_count=1e3)

[[np.float64(3.3033033033033092), np.float64(3.5035035035035094)],
 [np.float64(5.505505505505511), np.float64(5.705705705705711)],
 [np.float64(7.307307307307312), np.float64(7.507507507507512)],
 [np.float64(10.310310310310314), np.float64(10.510510510510514)]]

In [92]:
def Newton(a, b, coefs, epsilon=1e-6):
    xk = a
    xk_1 = b

    count = 0

    while abs(xk - xk_1) >= epsilon or abs(np.polyval(coefs, xk)) >= epsilon:
        xk_1 = xk
        xk = xk - np.polyval(coefs, xk)/np.polyval(np.polyder(coefs), xk)
        xk = np.clip(xk, a, b)
        count += 1

    return xk
main_vectors = np.zeros((len(intervals)))
for interval_index in range(len(intervals)):
    main_vectors[interval_index] = Newton(*intervals[interval_index], coef)
print(latex_matrix(np.atleast_2d(np.round(main_vectors, 4)).T))

$
\left(\begin{matrix}
    3.3875 \\
    5.6635 \\
    7.3377 \\
    10.4112 \\
\end{matrix}\right)
$


In [93]:
main_vector = np.atleast_2d(main_vectors).T
main_vector

array([[ 3.38754919],
       [ 5.66347944],
       [ 7.33774909],
       [10.41122228]])

In [94]:
coefs

array([   26.8       ,  -256.2267    ,  1031.97237   , -1465.66094076])

In [95]:
def Gauss_Method(A, b):
    A_ = A.copy()    
    b_ = b.copy()    
    x = np.zeros_like(b)
    for col in range(1, A.shape[1]):
        for row in range(col, A.shape[0]):
            zeroiator = (- A_[row, col-1]/A_[col-1, col-1])
            A_[row] = A_[row] + A_[col-1] * zeroiator
            b_[row] = b_[row] + b_[col-1] * zeroiator

    for col in range(A.shape[1]-1, -1, -1):
        result = 0
        for row in range(A.shape[0]-1, col, -1):
            result += A_[col, row] * x[row]
        x[col] = (b_[col] - result)/A_[col, col]


    return x
y = Gauss_Method(P, main_vector)
x = M @ y
y, x

(array([[ 5.66347944],
        [ 7.33774909],
        [10.41122228],
        [ 6.14900756]]),
 array([[  41.6016845 ],
        [ 176.88424859],
        [-112.8947859 ],
        [   6.14900756]]))

In [96]:
check = np.linalg.eigvals(matrix)
check = np.atleast_2d(check).T
print(check)
print(latex_matrix(np.round(check, 4)))

[[10.41122228]
 [ 3.38754919]
 [ 5.66347944]
 [ 7.33774909]]
$
\left(\begin{matrix}
    10.4112 \\
    3.3875 \\
    5.6635 \\
    7.3377 \\
\end{matrix}\right)
$
